In [1]:
%run ./Model.ipynb 

C:\Users\T-Rex\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
eyes_width = 60
eyes_height = 33
mouth_width = 50
mouth_height = 33
face_width = 66
face_height = 85
num_classes = 2
channel = 1

In [3]:
img_input = Input(shape=(face_height,face_width,1))
#img_input = Input(shape=(face_height,face_width,1))
#img_input = Input(shape=(face_height,face_width,1))

x = conv2d(img_input,32,3,2,'valid',True,name='conv1')
x = conv2d(x,32,3,1,'valid',True,name='conv2')
x = conv2d(x,64,3,1,'valid',True,name='conv3')

x_11 = MaxPooling2D(3,strides=1,padding='valid',name='stem_br_11'+'_maxpool_1')(x)
x_12 = conv2d(x,96,3,1,'valid',True,name='stem_br_12')

x = Concatenate(axis=3, name = 'stem_concat_1')([x_11,x_12])

x_21 = conv2d(x,64,1,1,'same',True,name='stem_br_211')
x_21 = conv2d(x_21,64,[1,7],1,'same',True,name='stem_br_212')
x_21 = conv2d(x_21,64,[7,1],1,'same',True,name='stem_br_213')
x_21 = conv2d(x_21,96,3,1,'valid',True,name='stem_br_214')

x_22 = conv2d(x,64,1,1,'same',True,name='stem_br_221')
x_22 = conv2d(x_22,96,3,1,'valid',True,name='stem_br_222')

x = Concatenate(axis=3, name = 'stem_concat_2')([x_21,x_22])

x_31 = conv2d(x,192,3,1,'valid',True,name='stem_br_31')
x_32 = MaxPooling2D(3,strides=1,padding='valid',name='stem_br_32'+'_maxpool_2')(x)
x = Concatenate(axis=3, name = 'stem_concat_3')([x_31,x_32])

In [4]:
#Inception-ResNet-A modules
x = incresA(x,0.15,name='incresA_1')
x = incresA(x,0.15 ,name='incresA_2')
#x = incresA(x,0.15,name='incresA_3')
#x = incresA(x,0.15,name='incresA_4')

#35 × 35 to 17 × 17 reduction module.
x_red_11 = MaxPooling2D(3,strides=2,padding='valid',name='red_maxpool_1')(x)

x_red_12 = conv2d(x,384,3,2,'valid',True,name='x_red1_c1')

x_red_13 = conv2d(x,256,1,1,'same',True,name='x_red1_c2_1')
x_red_13 = conv2d(x_red_13,256,3,1,'same',True,name='x_red1_c2_2')
x_red_13 = conv2d(x_red_13,384,3,2,'valid',True,name='x_red1_c2_3')

x = Concatenate(axis=3, name='red_concat_1')([x_red_11,x_red_12,x_red_13])

#Inception-ResNet-B modules
x = incresB(x,0.1,name='incresB_1')
x = incresB(x,0.1,name='incresB_2')
#x = incresB(x,0.1,name='incresB_3')
#x = incresB(x,0.1,name='incresB_4')
#x = incresB(x,0.1,name='incresB_5')
#x = incresB(x,0.1,name='incresB_6')
#x = incresB(x,0.1,name='incresB_7')

#17 × 17 to 8 × 8 reduction module.
x_red_21 = MaxPooling2D(3,strides=2,padding='valid',name='red_maxpool_2')(x)

x_red_22 = conv2d(x,256,1,1,'same',True,name='x_red2_c11')
x_red_22 = conv2d(x_red_22,384,3,2,'valid',True,name='x_red2_c12')

x_red_23 = conv2d(x,256,1,1,'same',True,name='x_red2_c21')
x_red_23 = conv2d(x_red_23,256,3,2,'valid',True,name='x_red2_c22')

x_red_24 = conv2d(x,256,1,1,'same',True,name='x_red2_c31')
x_red_24 = conv2d(x_red_24,256,3,1,'same',True,name='x_red2_c32')
x_red_24 = conv2d(x_red_24,256,3,2,'valid',True,name='x_red2_c33')

x = Concatenate(axis=3, name='red_concat_2')([x_red_21,x_red_22,x_red_23,x_red_24])

#Inception-ResNet-C modules
x = incresC(x,0.2,name='incresC_1')
#x = incresC(x,0.2,name='incresC_3')
InceptionOut = incresC(x,0.2,name='incresC_2')




In [5]:
#TOP
x = GlobalAveragePooling2D(data_format='channels_last')(InceptionOut)
#x = MaxPooling2D(pool_size=(3,2), padding='valid', data_format='channels_last')(InceptionOut)
#x = Dense(num_classes, activation='softmax')(x)
x = Flatten()(x)

In [6]:
#x = Dense(2048, activation='relu')(x)



#classification_1 = Dense(2048, activation='relu')(x)
classification_1 = BatchNormalization(scale=False)(x)
classification_1 = Dense(512, activation='relu')(classification_1)
classification_1 = BatchNormalization(scale=False)(classification_1)
classification_1 = Dense(64, activation='relu')(classification_1)

classification_1d = Dropout(0.5)(classification_1)
HeadOutput = Dense(3, activation='softmax', name = 'Head')(classification_1d)

#classification_2 = Dense(2048, activation='relu')(x)
classification_2 = BatchNormalization(scale=False)(x)
classification_2 = Dense(512, activation='relu')(classification_2)
classification_2 = BatchNormalization(scale=False)(classification_2)
classification_2 = Dense(64, activation='relu')(classification_2)
classification_2d = Dropout(0.5)(classification_2)
MouthOutput = Dense(3, activation='softmax', name = 'Mouth')(classification_2d)

#Mouth help eyes

#classification_3 = Dense(2048, activation='relu')(x)

classification_3 = BatchNormalization(scale=False)(x)
classification_3 = Dense(512, activation='relu')(classification_3)
classification_3 = BatchNormalization(scale=False)(classification_3)
classification_3 = Dense(64, activation='relu')(classification_3)

#classification_3out = Concatenate()([classification_1,classification_2,classification_3])

classification_3d = Dropout(0.5)(classification_3)
EyesOutput = Dense(2, activation='softmax', name = 'Eyes')(classification_3d)

#classification_4 = Dense(2048, activation='relu')(x)
classification_4 = BatchNormalization(scale=False)(x)
classification_4 = Dense(512, activation='relu')(classification_4)
classification_4 = BatchNormalization(scale=False)(classification_4)
classification_4 = Dense(64, activation='relu')(classification_4)

classification_4out = Concatenate()([classification_1,classification_2,classification_4])
classification_4d = Dropout(0.5)(classification_4out)
TotalOutput = Dense(2, activation='softmax', name = 'Total')(classification_4d)

model = Model(inputs=[img_input], outputs=[
                                            HeadOutput
                                           , MouthOutput
                                           , EyesOutput
                                           ,TotalOutput
                                        ], name='inception_resnet_v2')
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 85, 66, 1)    0                                            
__________________________________________________________________________________________________
conv1conv2d (Conv2D)            (None, 42, 32, 32)   288         input_1[0][0]                    
__________________________________________________________________________________________________
conv1conv2dbn (BatchNormalizati (None, 42, 32, 32)   96          conv1conv2d[0][0]                
__________________________________________________________________________________________________
conv1conv2dact (Activation)     (None, 42, 32, 32)   0           conv1conv2dbn[0][0]              
__________________________________________________________________________________________________
conv2conv2

In [7]:
model.compile(loss = {
                    'Head': 'categorical_crossentropy',
                    'Mouth': 'categorical_crossentropy',
                    'Eyes': 'categorical_crossentropy',
                    'Total': 'categorical_crossentropy'
                    },
                    loss_weights = {
                            'Head': 1.5,
                            'Mouth': 1.5,
                            'Eyes': 1.0,
                            'Total': 1.5,
                            }
                  , optimizer = Adam(lr = 0.0001, decay=1e-6), metrics=['accuracy'])

### Load Label

In [1]:
# load label

import os
import numpy as np

def FindFile(path,keyword):
                                                                                    
    #path = '.\Dataset\Training_Evaluation_Dataset\Training Dataset\\'
    #path = '.\Dataset\Training_Evaluation_Dataset\Training Dataset'

    File = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if keyword in file :
                File.append(os.path.join(r, file))

    #for f in File:
    #    print(f)
    print(np.array(File).shape)
    
    return File

In [2]:
Head_AllLabelFile = FindFile('.\Dataset\Training_Evaluation_Dataset\Training Dataset','head.txt')
Eye_AllLabelFile = FindFile('.\Dataset\Training_Evaluation_Dataset\Training Dataset','eye.txt')
Mouth_AllLabelFile = FindFile('.\Dataset\Training_Evaluation_Dataset\Training Dataset','mouth.txt')
Total_AllLabelFile = FindFile('.\Dataset\Training_Evaluation_Dataset\Training Dataset','drowsiness.txt')

(356,)
(356,)
(356,)
(356,)


In [3]:
import os
import numpy as np

#path = '.\Dataset\Training_Evaluation_Dataset\Training Dataset\\'
path = '.\Dataset\Training_Evaluation_Dataset\Training Image'

AllImageFolder = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for directory in d:
        if ("nonsleepyCombination" in directory or "sleepyCombination" in directory or "slowBlinkWithNodding"  in directory or "yawning" in directory)   :
            #print(directory)
            AllImageFolder.append(os.path.join(r, directory))
        
for d in AllImageFolder:
    print(d)
np.array(AllImageFolder).shape

.\Dataset\Training_Evaluation_Dataset\Training Image\001\glasses\nonsleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\001\glasses\sleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\001\glasses\slowBlinkWithNodding
.\Dataset\Training_Evaluation_Dataset\Training Image\001\glasses\yawning
.\Dataset\Training_Evaluation_Dataset\Training Image\001\nightglasses\nonsleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\001\nightglasses\sleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\001\nightglasses\slowBlinkWithNodding
.\Dataset\Training_Evaluation_Dataset\Training Image\001\nightglasses\yawning
.\Dataset\Training_Evaluation_Dataset\Training Image\001\night_noglasses\nonsleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\001\night_noglasses\sleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\001\night_noglasses\slowBlinkWithNodding
.\Dataset\Training_Evaluation_Dataset\Trainin

(356,)

In [4]:
num_samples = 234866
testing_sample = 167168
validation_sample = 83789

In [5]:
Head_Testing_AllLabelFile = FindFile('.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset','head.txt')
Eye_Testing_AllLabelFile = FindFile('.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset','eye.txt')
Mouth_Testing_AllLabelFile = FindFile('.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset','mouth.txt')
Total_Testing_AllLabelFile = FindFile('.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset','drowsiness.txt')

(20,)
(20,)
(20,)
(20,)


In [6]:
import os
import numpy as np

#path = '.\Dataset\Training_Evaluation_Dataset\Training Dataset\\'
path = '.\Dataset\Training_Evaluation_Dataset\Testing Image'

AllImageFolderTesting = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for directory in d:
        if ("mix" in directory):
            #print(directory)
            AllImageFolderTesting.append(os.path.join(r, directory))    
            
AllImageFolderTesting = AllImageFolderTesting
for d in AllImageFolderTesting:
    print(d)
    
#llImageFolder = AllImageFolder[2:]
np.array(AllImageFolderTesting).shape

.\Dataset\Training_Evaluation_Dataset\Testing Image\004\004_glasses_mix
.\Dataset\Training_Evaluation_Dataset\Testing Image\004\004_nightglasses_mix
.\Dataset\Training_Evaluation_Dataset\Testing Image\004\004_nightnoglasses_mix
.\Dataset\Training_Evaluation_Dataset\Testing Image\004\004_noglasses_mix
.\Dataset\Training_Evaluation_Dataset\Testing Image\004\004_sunglasses_mix
.\Dataset\Training_Evaluation_Dataset\Testing Image\022\022_glasses_mix
.\Dataset\Training_Evaluation_Dataset\Testing Image\022\022_nightglasses_mix
.\Dataset\Training_Evaluation_Dataset\Testing Image\022\022_nightnoglasses_mix
.\Dataset\Training_Evaluation_Dataset\Testing Image\022\022_noglasses_mix
.\Dataset\Training_Evaluation_Dataset\Testing Image\022\022_sunglasses_mix
.\Dataset\Training_Evaluation_Dataset\Testing Image\026\026_glasses_mix
.\Dataset\Training_Evaluation_Dataset\Testing Image\026\026_nightglasses_mix
.\Dataset\Training_Evaluation_Dataset\Testing Image\026\026_nightnoglasses_mix
.\Dataset\Training

(20,)

In [7]:
'''
Head_Testing_AllLabelFile = Head_Testing_AllLabelFile[6:15]
print(len(Head_Testing_AllLabelFile))
Eye_Testing_AllLabelFile = Eye_Testing_AllLabelFile[6:15]
print(len(Eye_Testing_AllLabelFile))
Mouth_Testing_AllLabelFile = Mouth_Testing_AllLabelFile[6:15]
print(len(Mouth_Testing_AllLabelFile))
Total_Testing_AllLabelFile = Total_Testing_AllLabelFile[6:15]
print(len(Total_Testing_AllLabelFile))
AllImageFolderTesting = AllImageFolderTesting[6:15]
'''

'\nHead_Testing_AllLabelFile = Head_Testing_AllLabelFile[6:15]\nprint(len(Head_Testing_AllLabelFile))\nEye_Testing_AllLabelFile = Eye_Testing_AllLabelFile[6:15]\nprint(len(Eye_Testing_AllLabelFile))\nMouth_Testing_AllLabelFile = Mouth_Testing_AllLabelFile[6:15]\nprint(len(Mouth_Testing_AllLabelFile))\nTotal_Testing_AllLabelFile = Total_Testing_AllLabelFile[6:15]\nprint(len(Total_Testing_AllLabelFile))\nAllImageFolderTesting = AllImageFolderTesting[6:15]\n'

In [8]:
AllImageFolderTesting

['.\\Dataset\\Training_Evaluation_Dataset\\Testing Image\\004\\004_glasses_mix',
 '.\\Dataset\\Training_Evaluation_Dataset\\Testing Image\\004\\004_nightglasses_mix',
 '.\\Dataset\\Training_Evaluation_Dataset\\Testing Image\\004\\004_nightnoglasses_mix',
 '.\\Dataset\\Training_Evaluation_Dataset\\Testing Image\\004\\004_noglasses_mix',
 '.\\Dataset\\Training_Evaluation_Dataset\\Testing Image\\004\\004_sunglasses_mix',
 '.\\Dataset\\Training_Evaluation_Dataset\\Testing Image\\022\\022_glasses_mix',
 '.\\Dataset\\Training_Evaluation_Dataset\\Testing Image\\022\\022_nightglasses_mix',
 '.\\Dataset\\Training_Evaluation_Dataset\\Testing Image\\022\\022_nightnoglasses_mix',
 '.\\Dataset\\Training_Evaluation_Dataset\\Testing Image\\022\\022_noglasses_mix',
 '.\\Dataset\\Training_Evaluation_Dataset\\Testing Image\\022\\022_sunglasses_mix',
 '.\\Dataset\\Training_Evaluation_Dataset\\Testing Image\\026\\026_glasses_mix',
 '.\\Dataset\\Training_Evaluation_Dataset\\Testing Image\\026\\026_nightgla

In [9]:
import numpy as np
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import cv2
from math import ceil

from keras.callbacks import EarlyStopping

import random



def LoadImageandLabel(Flag,batch_size,AllVideoImageFolder,Head_AllLabelFile,Mouth_AllLabelFile,Eye_AllLabelFile,Total_AllLabelFile):
    #print(Originalpath)
    count=0
    
    while True:
        LabelFileindex = 0
        

        Face_ImageDataList = []
        
        TotalLabelList = []
        HeadLabelList = []
        EyeLabelList = []
        MouthLabelList = []
        
        for Originalpath in AllVideoImageFolder:
            
            #=====load all label in a video
            HeadlabelPath = Head_AllLabelFile[LabelFileindex]
            MouthlabelPath = Mouth_AllLabelFile[LabelFileindex]
            EyelabelPath = Eye_AllLabelFile[LabelFileindex]
            TotallabelPath = Total_AllLabelFile[LabelFileindex]
            
            LabelFileindex = LabelFileindex + 1
            
                
            inputfile = open(HeadlabelPath, 'r').read()
            HeadTempLabelList = list(map(int, inputfile))
            inputfile = open(MouthlabelPath, 'r').read()
            MouthTempLabelList = list(map(int, inputfile))
            inputfile = open(EyelabelPath, 'r').read()
            EyeTempLabelList = list(map(int, inputfile))
            inputfile = open(TotallabelPath, 'r').read()
            TotalTempLabelList = list(map(int, inputfile))

            #=====find all files in path=======
            path = Originalpath+"\\face"
            Facefiles = []
            # r=root, d=directories, f = files
            for r, d, f in os.walk(path):
                for file in f:
                    if ".jpg" and 'frame_Middle' in file and not "ALL" in file:
                        temp = os.path.join(r, file)
                        Facefiles.append(temp)
            
            Facefiles.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
            Facefiles=Facefiles[:-2]
            
            #print(Originalpath)
            #print(len(TempLabelList))
            
            for FileIndex in range(len(Facefiles)):
                #======================load label
                
                index = Facefiles[FileIndex].find('frame_Middle')
                
                framenumber =int(Facefiles[FileIndex][index+12:-4]) 
                                
                HeadLabelarray = to_categorical(HeadTempLabelList[framenumber-1], num_classes=3)
                HeadLabelList.append(HeadLabelarray)
                
                MouthLabelarray = to_categorical(MouthTempLabelList[framenumber-1], num_classes=3)
                MouthLabelList.append(MouthLabelarray)
                
                EyeLabelarray = to_categorical(EyeTempLabelList[framenumber-1], num_classes=2)
                EyeLabelList.append(EyeLabelarray)

                TotalLabelarray = to_categorical(TotalTempLabelList[framenumber-1], num_classes=2)
                TotalLabelList.append(TotalLabelarray)

                #================================
                ImgGray = cv2.imread(Facefiles[FileIndex],0)
                
                #data augmentation
                if((random.uniform(0, 1)>0.5) and Flag == 0):
                    ImgGray = cv2.flip(ImgGray,1)
                    
                ImgGray = ImgGray.reshape(ImgGray.shape[0],ImgGray.shape[1],1)
                

                    
                Face_ImageDataList.append(ImgGray)
               
                count = count +1
                
                if count>=batch_size:
                    
                        yield ([np.array(Face_ImageDataList)],
                                                               [np.array(HeadLabelList),
                                                               np.array(MouthLabelList),
                                                               np.array(EyeLabelList), 
                                                               np.array(TotalLabelList)
                                                               ]
                                                               )
                        count=0
                        Face_ImageDataList = []
                        HeadLabelList = []
                        EyeLabelList = []
                        MouthLabelList = []
                        TotalLabelList = []
                
        #return Face_ImageDataList,HeadLabelList,MouthLabelList,EyeLabelList
                

C:\Users\T-Rex\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [10]:
'''
List1,List2,List3,List4 = LoadImageandLabelTesting(200,AllImageFolderTesting,Head_Testing_AllLabelFile,Mouth_Testing_AllLabelFile,Eye_Testing_AllLabelFile,Total_Testing_AllLabelFile)
print(len(List4))
print(len(List3))
print(len(List2))
print(len(List1))
'''

'\nList1,List2,List3,List4 = LoadImageandLabelTesting(200,AllImageFolderTesting,Head_Testing_AllLabelFile,Mouth_Testing_AllLabelFile,Eye_Testing_AllLabelFile,Total_Testing_AllLabelFile)\nprint(len(List4))\nprint(len(List3))\nprint(len(List2))\nprint(len(List1))\n'

In [11]:
import tensorflow as tf
%run ./Model.ipynb 
model = tf.keras.models.model_from_json(open('.\\Temp\\backup\\MultiTask_Model_11.json').read())
model.load_weights('.\\Temp\\backup\\MultiTask_Model_11.h5')

In [12]:
layer_dict = dict([(layer.name, layer) for layer in model.layers])
layer_dict

{'input_1': <tensorflow.python.keras.engine.input_layer.InputLayer at 0x20edf4da940>,
 'conv1conv2d': <tensorflow.python.keras.layers.convolutional.Conv2D at 0x20edf4dac50>,
 'conv1conv2dbn': <tensorflow.python.keras.layers.normalization.BatchNormalization at 0x20edf4dadd8>,
 'conv1conv2dact': <tensorflow.python.keras.layers.core.Activation at 0x20edf4dae80>,
 'conv2conv2d': <tensorflow.python.keras.layers.convolutional.Conv2D at 0x20edf4ea1d0>,
 'conv2conv2dbn': <tensorflow.python.keras.layers.normalization.BatchNormalization at 0x20edf4ea240>,
 'conv2conv2dact': <tensorflow.python.keras.layers.core.Activation at 0x20edf4ea278>,
 'conv3conv2d': <tensorflow.python.keras.layers.convolutional.Conv2D at 0x20edf4ea438>,
 'conv3conv2dbn': <tensorflow.python.keras.layers.normalization.BatchNormalization at 0x20edf4ea908>,
 'conv3conv2dact': <tensorflow.python.keras.layers.core.Activation at 0x20edf4ea940>,
 'stem_br_12conv2d': <tensorflow.python.keras.layers.convolutional.Conv2D at 0x20edf4e

In [13]:
'''
for layer in model.layers[:]:
    layer.trainable = False
    print(layer)
    
layer_dict['Total'].trainable = True
layer_dict['dropout_3'].trainable = True
layer_dict['concatenate'].trainable = True
layer_dict['dense_7'].trainable = True
layer_dict['batch_normalization_7'].trainable = True
layer_dict['dense_6'].trainable = True
layer_dict['batch_normalization_6'].trainable = True

layer_dict['Eyes'].trainable = True
layer_dict['dropout_2'].trainable = True
layer_dict['dense_5'].trainable = True
layer_dict['batch_normalization_5'].trainable = True
layer_dict['dense_4'].trainable = True
layer_dict['batch_normalization_4'].trainable = True
'''

"\nfor layer in model.layers[:]:\n    layer.trainable = False\n    print(layer)\n    \nlayer_dict['Total'].trainable = True\nlayer_dict['dropout_3'].trainable = True\nlayer_dict['concatenate'].trainable = True\nlayer_dict['dense_7'].trainable = True\nlayer_dict['batch_normalization_7'].trainable = True\nlayer_dict['dense_6'].trainable = True\nlayer_dict['batch_normalization_6'].trainable = True\n\nlayer_dict['Eyes'].trainable = True\nlayer_dict['dropout_2'].trainable = True\nlayer_dict['dense_5'].trainable = True\nlayer_dict['batch_normalization_5'].trainable = True\nlayer_dict['dense_4'].trainable = True\nlayer_dict['batch_normalization_4'].trainable = True\n"

In [14]:
model.compile(loss = {
                    'Head': 'categorical_crossentropy',
                    'Mouth': 'categorical_crossentropy',
                    'Eyes': 'categorical_crossentropy',
                    'Total': 'categorical_crossentropy'
                    },
                    loss_weights = {
                            'Head': 1.5,
                            'Mouth': 1.5,
                            'Eyes': 0.0,
                            'Total': 1.5,
                            }
                  , optimizer = Adam(lr = 0.000005, decay=1e-6), metrics=['accuracy'])


In [15]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 85, 66, 1)    0                                            
__________________________________________________________________________________________________
conv1conv2d (Conv2D)            (None, 42, 32, 32)   288         input_1[0][0]                    
__________________________________________________________________________________________________
conv1conv2dbn (BatchNormalizati (None, 42, 32, 32)   96          conv1conv2d[0][0]                
__________________________________________________________________________________________________
conv1conv2dact (Activation)     (None, 42, 32, 32)   0           conv1conv2dbn[0][0]              
__________________________________________________________________________________________________
conv2conv2

In [16]:
'''
from keras.utils.vis_utils import model_to_dot
from tensorflow.keras.utils import plot_model
from IPython.display import SVG

SVG(model_to_dot(model).create(prog='dot', format='svg'))
plot_model(model, to_file='model_plot.png', 
                  show_shapes=True, show_layer_names=True)
                  '''

"\nfrom keras.utils.vis_utils import model_to_dot\nfrom tensorflow.keras.utils import plot_model\nfrom IPython.display import SVG\n\nSVG(model_to_dot(model).create(prog='dot', format='svg'))\nplot_model(model, to_file='model_plot.png', \n                  show_shapes=True, show_layer_names=True)\n                  "

In [17]:
best = 0
second = 0

batch_size = 120

for i in range(200):
    model.fit_generator(LoadImageandLabel(0,batch_size,AllImageFolder,Head_AllLabelFile,Mouth_AllLabelFile
                                          ,Eye_AllLabelFile,Total_AllLabelFile),
                    steps_per_epoch=ceil(num_samples / batch_size),
                    epochs=1,
                    #callbacks=[earlyStopping],
                    workers = 5,
                    shuffle=True
                    )
    result= model.evaluate_generator(LoadImageandLabel(1,150,AllImageFolderTesting,Head_Testing_AllLabelFile,Mouth_Testing_AllLabelFile
                                          ,Eye_Testing_AllLabelFile,Total_Testing_AllLabelFile), steps=ceil(validation_sample / 500),workers=5,verbose = 1)
    
    
    print(result)
    if(result[8]>best):
        print("save model")
        best = result[8]
        model_json = model.to_json()
        open('MultiTask_Model_0821Best.json', 'w').write(model_json)
        model.save_weights('MultiTask_Model_0821Best.h5')
    elif(result[8]>second):
        print("save model2")
        second = result[8]
        model_json = model.to_json()
        open('MultiTask_Model_0821Second.json', 'w').write(model_json)
        model.save_weights('MultiTask_Model_0821Second.h5')

Epoch 1/1
168/168 [==============================] - 15s 89ms/step
[2.924879912109602, 0.31822161543739486, 0.4940704665752405, 1.1720944581195778, 1.1376278507966726, 0.896587301799584, 0.8406746028035524, 0.5331349206001809, 0.5333333332701364]
save model
Epoch 1/1
168/168 [==============================] - 13s 78ms/step
[3.140131139152107, 0.35791201995046285, 0.5215368641999832, 1.2093332096438167, 1.213971869765027, 0.8823809517946627, 0.8186507937054903, 0.519007935909377, 0.5495634908854429]
save model
Epoch 1/1
168/168 [==============================] - 13s 78ms/step
[3.1735864680792605, 0.36341833972860643, 0.508530605662253, 1.1902821259282064, 1.2437753683244366, 0.8765873027898904, 0.822103174602879, 0.5356349220687878, 0.5117063510420156]
save model2
Epoch 1/1
168/168 [==============================] - 13s 78ms/step
[3.0377588719129562, 0.3353343269546138, 0.5268102420768195, 1.1820328962390445, 1.1630280148903174, 0.8953968263452962, 0.8127380974280337, 0.536984126188881,

168/168 [==============================] - 13s 78ms/step
[3.6399354731575366, 0.3955066960172341, 0.6559087306704896, 1.2110668302228165, 1.37520822658137, 0.8803174599694709, 0.7523809531703591, 0.5244047631221336, 0.5341666665072331]
Epoch 1/1
168/168 [==============================] - 13s 78ms/step
[3.862276762191738, 0.4220030167953362, 0.6868901393008638, 1.1874809941689906, 1.4659580083902082, 0.8735317456580344, 0.7328174605998876, 0.523769841595952, 0.5329761921949241]
Epoch 1/1
168/168 [==============================] - 13s 78ms/step
[3.7495700101855967, 0.3980620074784506, 0.6696132264765045, 1.1800103615886266, 1.4320381086075766, 0.8786111110599623, 0.7503571446120206, 0.5346031755020487, 0.5505555571899527]
save model
Epoch 1/1
168/168 [==============================] - 13s 78ms/step
[3.8747049916890406, 0.4544556865324361, 0.7092222575636351, 1.1871892102644779, 1.4194587272414494, 0.848095239411729, 0.7073809523523474, 0.5445634934918157, 0.54853174707941]
save model2
Ep

168/168 [==============================] - 13s 78ms/step
[3.869841662945137, 0.38355287600676624, 0.6971218158794678, 1.189679617195257, 1.4992197498975743, 0.9001587292268163, 0.7489682548635063, 0.5283730138436935, 0.5503968264951947]
Epoch 1/1
168/168 [==============================] - 13s 78ms/step
[4.597179535088972, 0.4599808882227598, 0.8786477781365236, 1.23287663134397, 1.726157686361278, 0.8719841275763299, 0.6809523816087416, 0.5110317465095293, 0.5340079372926128]
Epoch 1/1
168/168 [==============================] - 13s 78ms/step
[4.2277469682462865, 0.40911295363660766, 0.7889380119032559, 1.2181190603434862, 1.620446998219926, 0.8885317446575278, 0.7301984127677444, 0.5152380938442158, 0.5406746023301301]
Epoch 1/1
168/168 [==============================] - 13s 78ms/step
[4.238962101572681, 0.42808722914738767, 0.8227966764562997, 1.1952481086898064, 1.575090809389838, 0.8868650798020619, 0.7303968246837723, 0.5332539684715725, 0.5652380958587552]
save model2
Epoch 1/1
16

KeyboardInterrupt: 